In [65]:
import numpy as np
from scipy.optimize import minimize

In [66]:
#Number of buses and lines
num_buses = 14
num_lines = 20

#incidence matrix
incidence_matrix = np.zeros((num_buses, num_lines))

#transmission lines
lines = [(1, 2), (1, 5), (2,3), (2,4), (2,5), (3,4),(4,5), (4,7), (4,9), (5,6), (6,11)
        ,(6,12), (6,13), (7,8), (7,9), (9,10), (9,14), (10,11), (12,13), (13,14)]

for line_idx, (from_bus, to_bus) in enumerate(lines):
    incidence_matrix[from_bus - 1, line_idx] = 1  # "from" bus
    incidence_matrix[to_bus - 1, line_idx] = -1   # "to" bus

In [67]:
num_buses = num_buses
Generator_number = [1,2,3,6,8]
num_generator = len(Generator_number)
#admittance matrix
Y = np.zeros((num_buses, num_buses), dtype=complex)

# Specified line parameters (example values)
specified_lines = lines
specified_R = [0.01938, 0.05403, 0.04699, 0.05811, 0.05695, 0.06701, 0.01335, 0, 0, 0, 0.09498, 0.12291
              ,0.06615, 0, 0, 0.03181, 0.12711, 0.08205, 0.22092, 0.17093]
specified_X = [0.05917, 0.22304, 0.19797, 0.17632, 0.17388, 0.17103, 0.04211, 0.20912, 0.55618, 0.25202
              ,0.19890, 0.25581, 0.13027, 0.17615, 0.11001, 0.0845, 0.27038, 0.19207, 0.19988, 0.34802]
specified_B = [0.0264, 0.0219, 0.0187, 0.0246, 0.017, 0.0173, 0.0064, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
               , 0, 0]

for (i, j), R_ij, X_ij, B_ij in zip(specified_lines, specified_R, specified_X, specified_B):
    Z_ij = R_ij + 1j * X_ij
    Y_ch_ij = 1j * B_ij
    Y[i-1, i-1] += Y_ch_ij
    Y[i-1, j-1] -= 1 / Z_ij
    Y[j-1, i-1] = np.conj(Y[i-1, j-1])

In [68]:
G = np.real(Y)
B = np.imag(Y)

In [69]:
V_magnitudes = [1.06, 1.045, 1.01, 1.01767, 1.01951, 1.07000, 1.06152, 1.09000, 1.05593, 1.05099, 1.05691
               ,1.05519, 1.05038, 1.03558]  # Example magnitudes for buses 1, 2, 3
theta_angles_degrees = [0, -4.9826, -12.725, -10.3128, -8.7738, -14.2209, -13.3596, -13.3596, -14.9385
                       ,-15.0972, -14.7906, -15.0755, -15.1562, -16.0336]  # Example phase angles in degrees for buses 1, 2, 3

theta_angles = np.deg2rad(theta_angles_degrees)

real_v = np.zeros(len(V_magnitudes))
imaginary_v = np.zeros(len(V_magnitudes))

for i in range(len(V_magnitudes)):
    real_v[i] = V_magnitudes[i] * np.cos(theta_angles[i])
    imaginary_v[i] = V_magnitudes[i] * np.sin(theta_angles[i])

print("Real parts of Voltage:", real_v)
print("Imaginary parts of Voltage:", imaginary_v)

Real parts of Voltage: [1.06       1.04105107 0.98519291 1.00122961 1.00757993 1.03721071
 1.03279425 1.06050356 1.02024282 1.01471546 1.02188959 1.01887451
 1.01384426 0.99529582]
Imaginary parts of Voltage: [ 0.         -0.0907616  -0.22247456 -0.18218533 -0.15550986 -0.26285727
 -0.24527686 -0.2518575  -0.27219985 -0.27373805 -0.26981553 -0.2744461
 -0.27462331 -0.28602825]


In [70]:
def calculate_phase_angle_difference(theta_i, theta_j):
    return theta_i - theta_j

theta_values = theta_angles_degrees

theta_values_radians = np.deg2rad(theta_values)

num_buses = len(theta_values)

theta_ij_matrix = np.zeros((num_buses, num_buses))

for i in range(num_buses):
    for j in range(num_buses):
        theta_ij_matrix[i, j] = calculate_phase_angle_difference(theta_values_radians[i], theta_values_radians[j])

In [71]:
PDi = [0, 21.7, 94.2, 47.8, 7.6, 11.2, 0, 0, 29.5, 9.0, 3.5, 6.1, 13.5,14.9]
QDi = [0, 12.7, 19.0, -3.9, 1.6, 7.5, 0, 0, 16.6, 5.8, 1.8, 1.6, 5.8, 5.0]
MVArating = [140, 70, 60, 50, 50, 50, 50, 25, 20, 50, 20, 20, 20, 20, 40, 20, 20, 20, 20, 20]
bounds_PG = [(0,332.4), (0, 140), (0, 100), (0, 100), (0, 100)]
bounds_Vi = [(1.0388, 1.0812000000000002), (1.0241, 1.0658999999999998), (0.9898, 1.0302), (0.9973166000000001, 1.0380234000000002), (0.9991197999999999, 1.0399002), (1.0486, 1.0914000000000001), (1.0402896, 1.0827504000000001), (1.0682, 1.1118000000000001), (1.0348114, 1.0770486000000001), (1.0299702000000002, 1.0720098), (1.0357718, 1.0780482), (1.0340862, 1.0762938000000002), (1.0293724, 1.0713876000000002), (1.0148684, 1.0562916)]
gi_up = [50, 20, 10, 10, 10]
gi_down = [-50, -20, -10, -10, -10]
a = [0.043, 0.25, 0.01, 0.01, 0.01]
b = [20, 20, 40, 40, 40]
c = [0, 0, 0, 0, 0]

In [73]:
def PQVIO(V, G, B, theta):
    num_buses = len(V)
    P = np.zeros(num_buses)
    Q = np.zeros(num_buses)
    
    for i in range(num_buses):
        for j in range(num_buses):
            P[i] += V[i] * (V[j] * (G[i, j] * np.cos(theta[i, j]) + B[i, j] * np.sin(theta[i, j])))
            Q[i] += V[i] * (V[j] * (G[i, j] * np.cos(theta[i, j]) - B[i, j] * np.sin(theta[i, j])))
    return P, Q

PQVO = PQVIO(V_magnitudes, G, B, theta_ij_matrix)
P_constraint = PQVO[0] + PDi
Q_constraint = PQVO[1] + QDi

P_constraints = [P_constraint[i-1] for i in Generator_number]
Q_constraints = [Q_constraint[i-1] for i in Generator_number]

def objective_function(x):
    PG = x[:num_generator]
    G_C = sum(a[i] * PG[i]**2 + b[i] * PG[i] + c[i] for i in range(num_generator))  
    return G_C 

def constraint_PG(x):
    PG = x[:num_generator]
    return [PG[i] - P_constraints[i] for i in range(num_generator)]

def constraint_QG(x):
    QG = x[:num_generator]
    return [QG[i] - Q_constraints[i] for i in range(num_generator)]

               
constraints = [{'type': 'ineq', 'fun': constraint_PG},
              {'type': 'ineq', 'fun': constraint_QG}]

bounds = bounds_PG #+ bounds_Vi


initial_guess = [30.0] * num_generator
result = minimize(objective_function, initial_guess, bounds=bounds, constraints=constraints)

optimal_PG = result.x[:num_generator]
optimal_Vi = result.x[num_generator*4:]
min_generation_cost = result.fun

print(f"Optimal Generator Values (PGi): {optimal_PG}")
print(f"Minimum Generation Cost (G(C)): {min_generation_cost}")

Optimal Generator Values (PGi): [ 0.         14.42229336 91.43414755  4.40640381  0.        ]
Minimum Generation Cost (G(C)): 4257.8647550964815
